<a href="https://colab.research.google.com/github/7ZXU/AI/blob/main/KoreaCarObjectDetection/carNplateObjectDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline 
# 브라우저 내부(inline)에서 rich output(그림, 소리, 애니메이션)을 바로 볼 수 있음

from IPython.display import Image # show image
from tqdm.auto import tqdm # 진행바 표시
import shutil as sh # 파일 이동
from PIL import Image

# **Loading Data**

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/TNT/KoreanCarObject/Dataset_for_LPR/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/TNT/KoreanCarObject/Dataset_for_LPR/test.csv')

In [ ]:
train_data

In [ ]:
test_data

# **Chage to YOLO dataset**

In [ ]:
# # 열 이름 : iamge -> image_id
# bb_df.rename(columns = {'image' : 'image_id'}, inplace = True)
# bb_df['image_id'] = bb_df['image_id'].apply(lambda x: x.split('.')[0]) # .jpg 떼고 id 저장

# # 바운딩박스 중심점 (x_center, y_center)
# # 바운딩박스 크기 (w, h)
# bb_df['x_center'] = (bb_df['xmin']+bb_df['xmax'])/2
# bb_df['y_center'] = (bb_df['ymin']+bb_df['ymax'])/2
# bb_df['w'] = bb_df['xmax'] - bb_df['xmin']
# bb_df['h'] = bb_df['ymax'] - bb_df['ymin']

# # 클래스 열 추가하고 0으로 초기화
# bb_df['classes'] = 0

# # 상위 행 출력
# bb_df.head()

In [ ]:
train_data.columns

In [ ]:
# train data
# car 와 plate 정보 분리 

train_car_data = train_data[['image_id', 'car_bb_xmin', 'car_bb_ymin', 'car_bb_w', 'car_bb_h']]
train_car_data

In [ ]:
train_plate_data = train_data[['image_id', 'plate_bb_xmin', 'plate_bb_ymin', 'plate_bb_w', 'plate_bb_h']]
train_plate_data

In [ ]:
# test data
# car, plate data 분리 

test_car_data = test_data[['image_id', 'car_bb_xmin', 'car_bb_ymin', 'car_bb_w', 'car_bb_h']]
test_plate_data = test_data[['image_id', 'plate_bb_xmin', 'plate_bb_ymin', 'plate_bb_w', 'plate_bb_h']]

In [ ]:
test_car_data.shape, test_plate_data.shape

In [ ]:
img_w = 1920
img_h = 1080

def get_yolo_input(data, class_name):
    # column rename
    data.columns = ['image_id', 'xmin', 'ymin', 'w', 'h']

    # 중심점 구하기 
    data['x_center'] = data['xmin'] + data['w']/2
    data['y_center'] = data['ymin'] + data['h']/2

    # 이미지 정규화 
    data['x_center'] = data['x_center']/img_w
    data['y_center'] = data['y_center']/img_h
    data['w'] = data['w']/img_w
    data['h'] = data['h']/img_h
    
    # 이미지 클래스 인덱싱
    # car(0) plate(1) 
    if class_name == 'car':
        data['class'] = 0
    elif class_name == 'plate':
        data['class'] = 1

    return data

In [ ]:
# 각 데이터를 yolo 를 위한 데이터로 변환
train_car_data_yolo = get_yolo_input(train_car_data, 'car') 
train_plate_data_yolo = get_yolo_input(train_plate_data, 'plate')
test_car_data_yolo = get_yolo_input(test_car_data, 'car') 
test_plate_data_yolo = get_yolo_input(test_plate_data, 'plate')

In [ ]:
train_car_data_yolo

In [ ]:
test_plate_data_yolo

In [ ]:
# car data + plate data
yolo_train_data = pd.concat([train_car_data_yolo, train_plate_data_yolo])
yolo_test_data = pd.concat([test_car_data_yolo, test_plate_data_yolo])

In [ ]:
yolo_test_data # car(0), plate(1) 로 구분

In [ ]:
base_path = '/content/drive/MyDrive/TNT/KoreanCarObject/LPRdata/labels'

def get_label_txt(type, data):
    for id, data in tqdm(data.groupby('image_id')):
        label_file_name = '{}/{}/{}.txt'.format(base_path, type, id)

        with open(label_file_name, 'w+') as f:
            row = data[['class','x_center', 'y_center', 'w', 'h']].astype(float).values # data의 각 열들을 float 형으로 받아옴
            row = row.astype(str) # 리스트 요소들 string 으로 변환
            for i in range(len(row)):
                text = ' '.join(row[i]) # 리스트의 값들을 공백 기준으로 합침
                f.write(text)
                f.write('\n') # 행 사이 줄바꿈      


함수 내용 확인

In [ ]:
# float 형으로 data 불러오기
row = yolo_train_data[['class','x_center', 'y_center', 'w', 'h']].astype(float).values
row

In [ ]:
# list 요소들을 string 으로 변환
row = row.astype(str)
row[0]

In [ ]:
# '구분자'.join(리스트)

# 리스트의 값과 값 사이에 구분자를 넣어서 하나의 문자열로 합쳐준다.
# '_'.join(['a', 'b', 'c']) 라 하면 "a_b_c" 와 같은 형태로 문자열을 만들어서 반환


text = ' '.join(row[0]) # 리스트 요소 공백으로 구분해서 합친 문자열 반환
text

label 저장

In [ ]:
get_label_txt('train', yolo_train_data)
get_label_txt('valid', yolo_test_data)

# **Training**

In [ ]:
%cd /content/drive/MyDrive/TNT/KoreanCarObject

In [ ]:
%%time

!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U pycocotools
!pip install -qr yolov5/requirements.txt  # install dependencies
!cp yolov5/requirements.txt ./

In [ ]:
%ls

In [ ]:
%cd /content/drive/MyDrive/TNT/KoreanCarObject/yolov5

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!python train.py --img 1920 --batch 8 --epoch 20 --data LPR.yaml --weights yolov5s.pt